In [1]:
import sys
sys.path.append("../agrilearn/submodules/commons/")
from agrilearn.commons.s3 import s3_utils
from agrilearn.crop_classification import evaluate_utils, yaml_utils, processing
import geopandas as gpd
import os
import pandas as pd

2025-04-23 12:56:10.343693: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-23 12:56:10.902399: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745412971.105759  938237 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745412971.169378  938237 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-23 12:56:11.687617: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## 1. Define Variables and Read Data

In [2]:
MAIN_PATH = "/agrilearn_app/output/experiment_12/data/"
GEOPACKAGE_RAW_PATH = os.path.join(MAIN_PATH, "geopackage/processed/SOYBEAN_48257_CORN_33809_SUGAR_CANE_17640_WHEAT_4226_COTTON_3178_RICE_925_test.gpkg")
EOPATCH_PATH_PROCESSED = os.path.join(MAIN_PATH, "eopatch/processed/")
REPORT_PATH = os.path.join(MAIN_PATH, "reports")

LABEL_EOPATCH_LOCATION = "eopath_location" 

In [3]:
df = gpd.read_file(GEOPACKAGE_RAW_PATH)

In [4]:
df.shape

(108035, 36)

In [5]:
df.head()

,monitoring_class,period,fonte,state,area,micro,start_season,end_season,peak_start,peak_end,...,is_valid_POS,is_valid_LOS,set_type,sampled_date,dataset_source,dataset_part,geometry_type,safra,edf,geometry
0,CORN,2022/2023,Agrosatélite - Grãos Sul 2022/2023,RS,None,490.0,2022-08-01,2023-07-01,2022-10-01,2023-04-01,...,True,False,train,dez_2024,/agrilearn_app/datasets/base/geopackage/proces...,test,MultiPolygon,safra,101,"MULTIPOLYGON (((-5872418.582 -3472913.558, -58..."
1,CORN,2022/2023,Agrosatélite - Grãos Sul 2022/2023,RS,184475.5792581472,491.0,2022-08-01,2023-07-01,2022-10-01,2023-04-01,...,True,False,train,dez_2024,/agrilearn_app/datasets/base/geopackage/proces...,test,MultiPolygon,safra,101,"MULTIPOLYGON (((-5776678.887 -3454619.446, -57..."
2,CORN,2022/2023,Agrosatélite - Grãos Sul 2022/2023,RS,306635.615729009,488.0,2022-08-01,2023-07-01,2022-10-01,2023-04-01,...,True,True,val,dez_2024,/agrilearn_app/datasets/base/geopackage/proces...,test,MultiPolygon,safra,101,"MULTIPOLYGON (((-5990899.154 -3423069.937, -59..."
3,CORN,2022/2023,Agrosatélite - Grãos Sul 2022/2023,RS,146966.765861118,489.0,2022-08-01,2023-07-01,2022-10-01,2023-04-01,...,True,False,train,dez_2024,/agrilearn_app/datasets/base/geopackage/proces...,test,MultiPolygon,safra,101,"MULTIPOLYGON (((-5953080 -3505253.46, -5953040..."
4,CORN,2022/2023,Agrosatélite - Grãos Sul 2022/2023,RS,84911.35149790395,500.0,2022-08-01,2023-07-01,2022-10-01,2023-04-01,...,True,True,train,dez_2024,/agrilearn_app/datasets/base/geopackage/proces...,test,MultiPolygon,safra,101,"MULTIPOLYGON (((-6133728.39 -3568966.103, -613..."


## 2. Processing Geopackage

### 2.1 Drop Columns NaN

In [6]:
shape_before = df.shape[1]
df.dropna(axis=1, how='all', inplace=True)
print(f"Removed Columns: {shape_before - df.shape[1]}, Percentage: {(shape_before - df.shape[1]) / shape_before * 100:.2f}%")

Removed Columns: 0, Percentage: 0.00%


### 2.2 Check NaN values

In [7]:
# Calcula a quantidade de registros nulos e a porcentagem de registros nulos
df_check_NaN = pd.concat([df.isna().sum(), df.isna().sum() / df.shape[0] * 100], axis=1)
df_check_NaN.columns = ['Null Count', 'NaN percentage']
df_check_NaN.sort_values('NaN percentage', ascending=False, inplace=True)
df_check_NaN.head()

,Null Count,NaN percentage
los_valid,41323,38.249641
eos_valid,41323,38.249641
pos_valid,41323,38.249641
sos_valid,41323,38.249641
is_valid_metrics,41323,38.249641


### 2.3 Transform all Geometry to Multipolygon

In [8]:
from shapely.geometry import Polygon, MultiPolygon

In [9]:
df['geometry'] = df['geometry'].apply(lambda geom: MultiPolygon([geom]) if isinstance(geom, Polygon) else geom)

### 2.4 Check Duplicates

In [14]:
subset_duplicate = ['geometry', 'period']

df['duplicates_id'] = None
duplicados = df[df.duplicated(subset=subset_duplicate, keep=False)]
# Adicione um identificador para os duplicados
duplicados['duplicates_id'] = duplicados.groupby('eopath_location').ngroup()
# Combine os dados duplicados com o DataFrame original
df.update(duplicados)
df.sort_values('duplicates_id', inplace=True)

In [15]:
duplicados.shape

(0, 37)

In [16]:
df[df['duplicates_id']==0]

,monitoring_class,period,fonte,state,area,micro,start_season,end_season,peak_start,peak_end,...,is_valid_LOS,set_type,sampled_date,dataset_source,dataset_part,geometry_type,safra,edf,geometry,duplicates_id


In [17]:
shape_before = df.shape[0]
df.drop_duplicates(subset=subset_duplicate, inplace=True)
print(f"Removed records: {shape_before - df.shape[0]}, Percentage: {(shape_before - df.shape[0]) / shape_before * 100:.2f}%")

Removed records: 0, Percentage: 0.00%


### 2.5 feature engineering

In [18]:
df['safra'] = df['period'].apply(lambda x: 'safrinha' if x.split('/')[0] == x.split('/')[1] else 'safra')

In [19]:
df['safra'].value_counts()

safra
safra       85800
safrinha    22235
Name: count, dtype: int64

## 3. Check processed eopatch

In [20]:
string_name = processing.get_geopackage_name(df)
string_name

'SOYBEAN_48257_CORN_33809_SUGAR_CANE_17640_WHEAT_4226_COTTON_3178_RICE_925'

#### A) PROCESSED

In [21]:
df_processed, df_overview_processed = processing.check_processed_eopatches_features_based_geopackage(df, 
                                                                                         eopatch_path=EOPATCH_PATH_PROCESSED,
                                                                                         label_eopatch_path=LABEL_EOPATCH_LOCATION)

2025-04-23 12:58:01,107 - INFO - Iniciando a execução da função 'check_processed_eopatches_features_based_geopackage'


  0%|          | 0/108035 [00:00<?, ?it/s]

2025-04-23 15:01:39,532 - INFO - Tempo de execução da função 'check_processed_eopatches_features_based_geopackage': 7418.42 segundos


In [22]:
df_overview_processed

,0
total_eopatches,108035.000000
eopatch_processed_exists,90341.000000
eopatches_not_exist,17694.000000
percentage_eopatches_not_exist,16.378026
len_data_exist,90341.000000
percentage_len_data_exist,83.621974
len_bands_exist,90341.000000
percentage_len_bands_exist,83.621974
len_time_interval_exist,90341.000000
percentage_len_time_interval_exist,83.621974


In [23]:
os.makedirs(REPORT_PATH, exist_ok=True)
final_processed_report_name = f"{REPORT_PATH}/overview_{string_name}_processed.csv"
df_overview_processed.to_csv(final_processed_report_name)
final_processed_report_name

'/agrilearn_app/output/experiment_12/data/reports/overview_SOYBEAN_48257_CORN_33809_SUGAR_CANE_17640_WHEAT_4226_COTTON_3178_RICE_925_processed.csv'

In [24]:
df_processed[df_processed['eopatch_processed_exists']==False]

,eopatch_location_id,eopatch_processed_exists,len_data,len_bands,len_time_interval,len_timestamp,error_processed
17644,start_2020-04-01_end_2021-04-01_monitoring_cla...,False,NaN,NaN,NaN,NaN,'bands'
17645,start_2021-04-01_end_2022-04-01_monitoring_cla...,False,NaN,NaN,NaN,NaN,'bands'
17646,start_2021-04-01_end_2022-04-01_monitoring_cla...,False,NaN,NaN,NaN,NaN,'bands'
17647,start_2020-04-01_end_2021-04-01_monitoring_cla...,False,NaN,NaN,NaN,NaN,'bands'
17648,start_2020-04-01_end_2021-04-01_monitoring_cla...,False,NaN,NaN,NaN,NaN,'bands'
...,...,...,...,...,...,...,...
104877,start_2023-01-01_end_2023-09-01_monitoring_cla...,False,NaN,NaN,NaN,NaN,'bands'
104878,start_2023-01-01_end_2023-09-01_monitoring_cla...,False,NaN,NaN,NaN,NaN,'bands'
104879,start_2023-01-01_end_2023-09-01_monitoring_cla...,False,NaN,NaN,NaN,NaN,'bands'
104887,start_2023-01-01_end_2023-09-01_monitoring_cla...,False,NaN,NaN,NaN,NaN,'bands'


In [25]:
df_processed[df_processed['eopatch_processed_exists']==True]

,eopatch_location_id,eopatch_processed_exists,len_data,len_bands,len_time_interval,len_timestamp,error_processed
0,start_2022-08-01_end_2023-07-01_monitoring_cla...,True,4.0,10.0,2.0,20.0,None
1,start_2022-08-01_end_2023-07-01_monitoring_cla...,True,4.0,10.0,2.0,19.0,None
2,start_2022-08-01_end_2023-07-01_monitoring_cla...,True,4.0,10.0,2.0,44.0,None
3,start_2022-08-01_end_2023-07-01_monitoring_cla...,True,4.0,10.0,2.0,23.0,None
4,start_2022-08-01_end_2023-07-01_monitoring_cla...,True,4.0,10.0,2.0,26.0,None
...,...,...,...,...,...,...,...
108030,start_2023-01-01_end_2023-09-01_monitoring_cla...,True,4.0,10.0,2.0,21.0,None
108031,start_2023-01-01_end_2023-09-01_monitoring_cla...,True,4.0,10.0,2.0,23.0,None
108032,start_2023-01-01_end_2023-09-01_monitoring_cla...,True,4.0,10.0,2.0,56.0,None
108033,start_2023-01-01_end_2023-09-01_monitoring_cla...,True,4.0,10.0,2.0,14.0,None


#### B) INPUT_MODEL

In [27]:
# df_input_model, df_overview_input = processing.check_input_model_eopatches_features_based_geopackage(df, 
#                                                                                                      eopatch_path=NEW_EOPATCH_PATH_INPUT_MODEL,
#                                                                                                      label_eopatch_path=LABEL_EOPATCH_LOCATION)

In [ ]:
# df_overview_input

In [ ]:
# df_input_model[(df_input_model['label_unique'].notna()) & 
#                 (df_input_model['geopackage_class'] != df_input_model['label_unique'])]

In [ ]:
# os.makedirs(REPORT_PATH, exist_ok=True)
# final_processed_report_name = f"{REPORT_PATH}/overview_{string_name}_input_model.csv"
# df_overview_input.to_csv(final_processed_report_name)
# final_processed_report_name

## 5. Merge Data and Save

### a) Processed Data

In [29]:
# Usando pd.merge para juntar os DataFrames
df_merged = pd.merge(df, 
                     df_processed, 
                     left_on=LABEL_EOPATCH_LOCATION,
                     right_on='eopatch_location_id', 
                     how='left')
df_merged.shape

(108035, 44)

In [32]:
result_string = processing.get_geopackage_name(df_merged, label_monitoring_class='monitoring_class')
GEOPACKAGE_PROCESSED_PATH = os.path.join(GEOPACKAGE_RAW_PATH.split('/raw/')[0]+"/", "processed", result_string+ ".gpkg")
GEOPACKAGE_PROCESSED_PATH

'/agrilearn_app/output/experiment_12/data/geopackage/processed/SOYBEAN_48257_CORN_33809_SUGAR_CANE_17640_WHEAT_4226_COTTON_3178_RICE_925_test.gpkg/processed/SOYBEAN_48257_CORN_33809_SUGAR_CANE_17640_WHEAT_4226_COTTON_3178_RICE_925.gpkg'

In [36]:
df_merged[df_merged['eopatch_processed_exists'] == False]['monitoring_class'].value_counts()

monitoring_class
SUGAR_CANE    17640
COTTON           54
Name: count, dtype: int64

### b) Input Model

In [30]:
# df_full = pd.merge(df_merged, df_input_model, left_on='eopath_location', right_on='eopatch_location_id', how='left')
# df_full.shape

In [31]:
# df_full.to_file(GEOPACKAGE_PROCESSED_PATH, 
#                 driver='GPKG', 
#                 engine='fiona')